In [2]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

#impore pipeline libraries
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
df_train_prepro = pd.read_csv('data/preprocessed_train_data.csv')
df_train_prepro

,Unnamed: 0,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration
0,0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-25.0,9300.0
1,1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-33.0,10200.0
2,2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-14.0,15600.0
3,3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-35.0,10200.0
4,4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-20.0,8700.0
...,...,...,...,...,...,...,...,...,...
99742,99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0
99743,99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0
99744,99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0
99745,99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0


In [4]:
departure_date_country = df_train_prepro[["departure_time", "departure_point"]].sort_values(by=["departure_time", "departure_point"])

In [ ]:
departure_date_country.to_csv("data/departure_date_country.csv")

In [5]:
departure_date_country

,departure_time,departure_point
4694,2016-01-01 00:15:00,NKC
47985,2016-01-01 01:20:00,DKR
10590,2016-01-01 06:15:00,TUN
42333,2016-01-01 06:30:00,TUN
40571,2016-01-01 07:00:00,MIR
...,...,...
35196,2018-12-31 22:00:00,TUN
37710,2018-12-31 22:00:00,TUN
99406,2018-12-31 22:20:00,DSS
33584,2018-12-31 22:40:00,ABJ


In [7]:
airports_coutry_city = pd.read_csv("data/airport_country_city_mapping.csv")
airports_coutry_city.head()

,IATA_Code,Country,City,2016 Passengers,2017 Passengers,2018 Passengers
0,AAE,Algeria,Annaba,"287,421","302,492","318,116"
1,AAL,Denmark,Aalborg,"1,518,879","1,607,987","1,643,303"
2,ABJ,Côte d’Ivoire,Abidjan,"1,860,000","2,080,000","2,200,000"
3,ADB,Turkey,Izmir,"11,951,183","12,400,000","13,100,000"
4,AHU,Morocco,Al Hoceima,"92,083","122,036","179,355"


In [8]:
df_train_prepro = df_train_prepro.merge(
    airports_coutry_city[['IATA_Code', 'Country', 'City']],
    left_on='departure_point',
    right_on='IATA_Code',
    how='left'
).drop(columns=['IATA_Code'])

In [9]:
df_train_prepro.head()

,Unnamed: 0,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,Country,City
0,0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-25.0,9300.0,Tunisia,Tunis
1,1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-33.0,10200.0,Tunisia,Djerba
2,2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-14.0,15600.0,Tunisia,Tunis
3,3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-35.0,10200.0,Turkey,Istanbul
4,4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-20.0,8700.0,Morocco,Casablanca


In [9]:
df_train_prepro.to_csv('data/data_with_cou_city.csv', index=False)

In [10]:
df_train_prepro

,Unnamed: 0,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,Country,City
0,0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-25.0,9300.0,Tunisia,Tunis
1,1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-33.0,10200.0,Tunisia,Djerba
2,2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-14.0,15600.0,Tunisia,Tunis
3,3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-35.0,10200.0,Turkey,Istanbul
4,4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-20.0,8700.0,Morocco,Casablanca
...,...,...,...,...,...,...,...,...,...,...,...
99742,99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,Tunisia,Tunis
99743,99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,France,Paris
99744,99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,Belgium,Brussels
99745,99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,France,Paris


In [ ]:
%pip install holidays

In [11]:
import pandas as pd
from datetime import datetime
import holidays

# Load your dataset
df = pd.read_csv("data/data_with_cou_city.csv")

# Ensure departure_time is parsed to datetime
df['departure_time'] = pd.to_datetime(df['departure_time'])
df['departure_date'] = df['departure_time'].dt.date

# Mapping for countries to ISO codes supported by the holidays package
country_code_map = {
    'United Kingdom': 'GB',
    'Côte d’Ivoire': 'CI',
    'DR Congo': 'CD',
    'Russia': 'RU',
    'Turkey': 'TR',
    'France': 'FR',
    'Germany': 'DE',
    'Italy': 'IT',
    'Spain': 'ES',
    'Canada': 'CA',
    'Portugal': 'PT',
    'Austria': 'AT',
    'Belgium': 'BE',
    'Switzerland': 'CH',
    'Greece': 'GR',
    'Netherlands': 'NL',
    'Luxembourg': 'LU',
    'Ireland': 'IE',
    'Denmark': 'DK',
    'Norway': 'NO',
    'Sweden': 'SE',
    'Finland': 'FI',
    'Czech Republic': 'CZ',
    'Poland': 'PL',
    'Slovakia': 'SK',
    'Slovenia': 'SI',
    'Hungary': 'HU',
    'Romania': 'RO',
    'Lithuania': 'LT',
    'Latvia': 'LV',
    'Estonia': 'EE',
    'Ukraine': 'UA',
    'Serbia': 'RS',
    'Bosnia and Herzegovina': 'BA'
}

# Build holiday sets for countries
country_holidays = {}

for country in df['Country'].dropna().unique():
    code = country_code_map.get(country)
    if code:
        country_holidays[country] = holidays.CountryHoliday(code, years=[2016, 2017, 2018])
    else:
        country_holidays[country] = set()

# Function to check if a date is a holiday in a given country
def is_holiday(row):
    country = row['Country']
    date = row['departure_date']
    return 1 if date in country_holidays.get(country, set()) else 0

# Apply function
df['is_holiday'] = df.apply(is_holiday, axis=1)

# Save the updated file
df.to_csv("data_with_holidays.csv", index=False)
print("✅ File saved as data_with_holidays.csv")


✅ File saved as data_with_holidays.csv


In [26]:
df["City"].unique()

array(['Tunis', 'Djerba', 'Istanbul', 'Casablanca', 'Paris', 'Abidjan',
       'Algiers', 'Lisbon', 'Monastir', 'Ouagadougou', 'Porto',
       'Barcelona', 'Milan', 'Marseille', 'Oujda', 'Berlin', 'Venice',
       'Strasbourg', 'Enfidha', 'Nice', 'Toulouse', 'Cairo', 'Düsseldorf',
       'London', 'Geneva', 'Lyon', 'Rome', 'Zurich', 'Nador', 'Frankfurt',
       'Brussels', 'Sfax', 'Beirut', 'Vienna', 'Bologna', 'Munich',
       'Oran', 'Nantes', 'Tozeur', 'Budapest', 'Stockholm', 'Copenhagen',
       'Amsterdam', 'Jeddah', 'Belgrade', 'Prague', 'Tangier',
       'Nouakchott', 'Gafsa', 'Naples', 'Medina', 'Bordeaux',
       'Al Hoceima', 'Ljubljana', 'Hamburg', 'Brno', 'Ostrava', 'Madrid',
       'Valletta', 'Bamako', 'Dakar', 'Gabès', 'Moscow', 'Erbil',
       'Bejaia', 'Montreal', 'Lomé', 'Cologne', 'Palermo', 'Niamey',
       'Lubumbashi', 'Tabarka', 'Pisa', 'Charleroi', 'Kigali', 'Ostend',
       'Franceville', 'Vilnius', 'Constantine', 'Ankara', 'Libreville',
       'Košice', 'Cona

In [28]:
df[(df["is_holiday"] == 1) & (df['City'] == 'Rome')].sort_values(by="departure_time")

,Unnamed: 0,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,Country,City,departure_date,is_holiday,holiday_length
38289,38289,FCO,TUN,2016-01-01 10:05:00,2016-01-01 11:20:00,ATA,TU 31BIMO,5.0,4500.0,Italy,Rome,2016-01-01,1,1
2214,2214,FCO,TUN,2016-01-01 16:05:00,2016-01-01 17:20:00,ATA,TU 31BIMQ,-24.0,4500.0,Italy,Rome,2016-01-01,1,1
48012,48012,FCO,TUN,2016-01-06 10:05:00,2016-01-06 11:20:00,ATA,TU 31BIMO,50.0,4500.0,Italy,Rome,2016-01-06,1,1
44121,44121,FCO,TUN,2016-03-27 10:20:00,2016-03-27 11:35:00,ATA,TU 32AIMH,20.0,4500.0,Italy,Rome,2016-03-27,1,2
42842,42842,FCO,TUN,2016-03-27 16:50:00,2016-03-27 18:05:00,ATA,TU 320IMR,129.0,4500.0,Italy,Rome,2016-03-27,1,2
2746,2746,FCO,TUN,2016-03-28 10:20:00,2016-03-28 11:35:00,ATA,TU 31BIMQ,-18.0,4500.0,Italy,Rome,2016-03-28,1,2
3032,3032,FCO,TUN,2016-03-28 16:50:00,2016-03-28 18:05:00,ATA,TU 32AIMH,-35.0,4500.0,Italy,Rome,2016-03-28,1,2
46477,46477,FCO,TUN,2016-04-25 10:20:00,2016-04-25 11:35:00,ATA,TU 31BIMQ,7.0,4500.0,Italy,Rome,2016-04-25,1,1
4024,4024,FCO,TUN,2016-04-25 16:50:00,2016-04-25 18:05:00,ATA,TU 32AIML,-26.0,4500.0,Italy,Rome,2016-04-25,1,1
38973,38973,FCO,TUN,2016-06-02 10:20:00,2016-06-02 11:35:00,ATA,TU 31BIMO,34.0,4500.0,Italy,Rome,2016-06-02,1,1


In [ ]:
from datetime import datetime

# Function to get the length of the consecutive holiday period for a given date and country
def holiday_length(row):
    if row['is_holiday'] != 1:
        return 0
    country = row['Country']
    date = row['departure_date']
    holidays_set = set(country_holidays.get(country, {}).keys())
    if not holidays_set:
        return 1  # If no holidays found, treat as single-day holiday

    # Convert date to datetime.date if it's a string
    if isinstance(date, str):
        date = datetime.strptime(date, "%Y-%m-%d").date()

    # Count consecutive days before and after
    length = 1
    prev_date = date
    next_date = date

    # Check previous days
    while True:
        prev_date = prev_date - pd.Timedelta(days=1)
        if prev_date in holidays_set:
            length += 1
        else:
            break

    # Check next days
    while True:
        next_date = next_date + pd.Timedelta(days=1)
        if next_date in holidays_set:
            length += 1
        else:
            break

    return length

# Apply the function to the DataFrame
df['holiday_length'] = df.apply(holiday_length, axis=1)

# Show a sample of holidays with their lengths
df[df['is_holiday'] == 1][['departure_date', 'Country', 'holiday_length']]

,departure_date,Country,holiday_length
525,2016-04-29,Serbia,5
1084,2016-04-23,Turkey,1
1111,2016-01-01,France,1
1901,2016-07-05,Czech Republic,2
1903,2016-07-05,Czech Republic,2


In [20]:
df[df["is_holiday"] == 1]

,Unnamed: 0,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,Country,City,departure_date,is_holiday,holiday_length
525,525,BEG,TUN,2016-04-29 17:25:00,2016-04-29 19:25:00,ATA,TU 32AIMC,-16.0,7200.0,Serbia,Belgrade,2016-04-29,1,5
1084,1084,IST,TUN,2016-04-23 23:25:00,2016-04-24 02:15:00,ATA,TU 32AIMI,-57.0,10200.0,Turkey,Istanbul,2016-04-23,1,1
1111,1111,ORY,TUN,2016-01-01 18:30:00,2016-01-01 21:00:00,ATA,TU 32AIMF,-10.0,9000.0,France,Paris,2016-01-01,1,1
1901,1901,BRQ,OSR,2016-07-05 17:05:00,2016-07-05 17:40:00,ATA,TU 31AIMK,-40.0,2100.0,Czech Republic,Brno,2016-07-05,1,2
1903,1903,OSR,MIR,2016-07-05 18:30:00,2016-07-05 21:00:00,ATA,TU 31AIMK,-8.0,9000.0,Czech Republic,Ostrava,2016-07-05,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99421,99421,LYS,TUN,2018-11-11 10:40:00,2018-11-11 12:35:00,ATA,TU 32AIMD,65.0,6900.0,France,Lyon,2018-11-11,1,1
99533,99533,VCE,TUN,2018-12-26 10:30:00,2018-12-26 12:10:00,ATA,TU 32AIMH,35.0,6000.0,Italy,Venice,2018-12-26,1,2
99556,99556,ORY,DJE,2018-11-01 10:15:00,2018-11-01 13:05:00,ATA,TU 32AIMI,12.0,10200.0,France,Paris,2018-11-01,1,1
99585,99585,FRA,TUN,2018-12-26 10:30:00,2018-12-26 12:55:00,ATA,TU 32AIMD,24.0,8700.0,Germany,Frankfurt,2018-12-26,1,2


In [22]:
df

,Unnamed: 0,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,Country,City,departure_date,is_holiday,holiday_length
0,0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-60.0,9300.0,Tunisia,Tunis,2016-01-16,0,0
1,1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-21.0,10200.0,Tunisia,Djerba,2016-01-17,0,0
2,2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-4.0,15600.0,Tunisia,Tunis,2016-01-20,0,0
3,3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-142.0,10200.0,Turkey,Istanbul,2016-01-21,0,0
4,4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-40.0,8700.0,Morocco,Casablanca,2016-01-22,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99742,99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,Tunisia,Tunis,2018-04-18,0,0
99743,99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,France,Paris,2018-12-05,0,0
99744,99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,Belgium,Brussels,2018-12-05,0,0
99745,99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,France,Paris,2018-12-04,0,0


In [21]:
df.to_csv("data_with_holidays_and_lengths.csv", index=False)